In [14]:
# 필요한 라이브러리 설치 및 인증 (최초 한 번 실행)
!pip install --upgrade gspread google-auth


In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.font_manager as fm
import warnings
from sentence_transformers import SentenceTransformer
from scipy.sparse import csr_matrix

warnings.simplefilter("ignore")

import logging
logging.getLogger("pdfminer.pdfpage").setLevel(logging.ERROR)

In [27]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

In [31]:

from google.colab import auth
auth.authenticate_user()

import gspread
from google.oauth2.service_account import Credentials

# Google Sheets API를 사용할 범위 지정
scopes = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']

# 기본 인증을 사용하는 경우
gc = gspread.authorize(google.auth.default(scopes=scopes)[0])

# 스프레드시트 열기 (문서 URL이나 제목을 사용)
spreadsheet = gc.open("구직자 응답")  # 스프레드시트 제목 사용
worksheet = spreadsheet.sheet1  # 첫 번째 시트 선택

# 데이터 가져오기
data = worksheet.get_all_records()
print(data)

[{'타임스탬프': '2025. 3. 29 오후 9:43:57', '직무': '저는 데이터 분석가 직무에 지원하고자 합니다. 데이터를 기반으로 인사이트를 도출하고, 비즈니스 의사결정에 도움을 주는 역할을 수행하고 싶습니다.', '경력': '저는 3년 동안 IT 기업에서 데이터 분석가로 근무했습니다. 주로 고객의 데이터를 분석하고, 트렌드 예측을 위해 다양한 분석 도구를 사용했습니다. 현재는 머신러닝 모델을 활용한 예측 분석 업무를 맡고 있습니다.', '자격증': "저는 데이터 분석 관련 자격증인 'Google Data Analytics Professional Certificate'를 보유하고 있습니다. 또한, 컴퓨터공학 학위를 취득했습니다.", '전공': '서울대학교에서 컴퓨터공학을 전공했으며, 학사 졸업 후 데이터 분석 관련 교육을 이수했습니다.', '기술': '저는 Python, SQL, R을 사용한 데이터 분석 기술을 보유하고 있습니다. 또한, 머신러닝 모델을 구축하고 분석하는 데 필요한 기술도 익혔습니다. 또한, Tableau를 사용한 데이터 시각화 경험도 있습니다.', '관심 있는 공고 인덱스': '3 8 45'}, {'타임스탬프': '2025. 3. 29 오후 9:45:57', '직무': '저는 데이터 분석가로서, 회사의 다양한 데이터를 분석하여 인사이트를 도출하고, 이를 통해 비즈니스 전략을 제시하는 역할을 맡고 싶습니다. ', '경력': '저는 2년 동안 데이터 분석 업무를 맡아왔습니다. 주로 SQL을 사용해 데이터를 추출하고, Python을 활용해 데이터를 분석하며, 결과를 시각화하는 업무를 했습니다. 현재는 마케팅 데이터 분석 팀에서 일하며, 고객 행동 패턴 분석을 통해 마케팅 전략을 제시하는 역할을 하고 있습니다.', '자격증': "저는 데이터 분석 관련 자격증인 'Google Data Analytics Professional Certificate'를 보유하고 있습니다. 또한, 컴퓨터공학 학위를 취득했습니다.", '전공': '저는 서울대학교

In [32]:
job_seekers = pd.DataFrame(data)
job_seekers

,타임스탬프,직무,경력,자격증,전공,기술,관심 있는 공고 인덱스
0,2025. 3. 29 오후 9:43:57,저는 데이터 분석가 직무에 지원하고자 합니다. 데이터를 기반으로 인사이트를 도출하고...,저는 3년 동안 IT 기업에서 데이터 분석가로 근무했습니다. 주로 고객의 데이터를 ...,저는 데이터 분석 관련 자격증인 'Google Data Analytics Profe...,"서울대학교에서 컴퓨터공학을 전공했으며, 학사 졸업 후 데이터 분석 관련 교육을 이수...","저는 Python, SQL, R을 사용한 데이터 분석 기술을 보유하고 있습니다. 또...",3 8 45
1,2025. 3. 29 오후 9:45:57,"저는 데이터 분석가로서, 회사의 다양한 데이터를 분석하여 인사이트를 도출하고, 이를...",저는 2년 동안 데이터 분석 업무를 맡아왔습니다. 주로 SQL을 사용해 데이터를 추...,저는 데이터 분석 관련 자격증인 'Google Data Analytics Profe...,"저는 서울대학교에서 통계학을 전공했으며, 석사 과정을 마친 후 데이터 분석 관련 업...","저는 Python과 R을 사용한 데이터 분석에 능숙하며, SQL을 이용한 데이터베이...",60 145 189 140 125 223
2,2025. 3. 29 오후 9:51:02,저는 군인을 희망합니다. 육군 부사관이나 장교로 근무하고 싶습니다.,"ROTC 리더십 교육을 수료했습니다. 리더십, 작전계획 수립 훈련 등을 이수했습니다.","한국사능력검정시험 2급, 한자 3급",군사학과 졸업,"체력단련 습관화, 단체생활 적응력 우수, 청렴성 및 책임감 강함",9 10 11 24 25 78
3,2025. 3. 29 오후 9:57:08,보건행정 및 병원 사무관리,"대학병원 원무팀에서 1년 근무하며 진료 예약, 입퇴원 수속 및 의료비 정산 업무 수행","병원행정사, 의료보험사",보건행정학과,"건강보험 청구, OCS(처방전달시스템), 전자의무기록(EMR) 시스템 사용 가능",45 46 48 49 54 212 161 162
4,2025. 3. 29 오후 9:58:30,건강보험 심사 및 병원 행정,"건강보험 심사평가원 실습 경험, 의원급 의료기관에서 원무과 아르바이트 경험","건강보험 청구사, 병원행정사",보건학과,"건강보험 심사 청구, 전자의무기록(EMR) 관리, 환자 상담 능력",49 50 161 162 163 232 212
5,2025. 3. 29 오후 10:00:18,공공기관 행정지원 및 사무 보조,"지방자치단체 행정팀 인턴 경험, 문서 정리 및 공문 작성 업무 수행","컴퓨터활용능력 2급, 워드프로세서 1급",행정학과,"문서 작성(한글, MS Office), 데이터 정리 및 보고서 작성, 전자결재 시스...",178 179 180 208 209 220 160 161
6,2025. 3. 29 오후 10:02:29,기업 인사 및 총무 지원,"중견기업 인사팀 아르바이트, 직원 근태 관리 및 급여 정산 보조 업무 수행","ERP 인사관리, 컴퓨터활용능력 1급",경영학과,"Excel을 활용한 데이터 관리, 인사 관련 서류 작성, 사무 행정 시스템 사용 가능",90 61 62 46 47


In [34]:
import re

# 구직자 프로필 텍스트 생성
def create_profile_text(row):
    profile = f"직무: {job_seekers.loc[row, '직무']} 경력: {job_seekers.loc[row, '경력']} 자격증: {job_seekers.loc[row, '자격증']} 전공: {job_seekers.loc[row, '전공']} 기술: {job_seekers.loc[row, '기술']}"
    return profile

# 1. user_text, 3. interaction_data : 구글폼에서 입력받기
user_text = create_profile_text(0)

input_str = job_seekers.loc[0, '관심 있는 공고 인덱스']


# 정규표현식을 사용하여 문자열에서 숫자(연속된 숫자들)를 추출합니다.
numbers = re.findall(r'\d+', input_str)

# 추출된 문자열 숫자들을 정수로 변환하고, ('user_1', 숫자) 형태의 튜플로 리스트를 생성합니다.
interaction_data = [('user_1', int(num)) for num in numbers]

print(interaction_data)

[('user_1', 3), ('user_1', 8), ('user_1', 45)]
